In [ ]:
# !pip install -r requirements.txt

In [2]:
import asyncio
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
import os
from pathlib import Path
from flask import Flask

# Define the path to your MCP server
server_params = StdioServerParameters(
    command="docker",
    args=[
        "run",
        "-i",
        "--rm",
        "-e",
        "GITHUB_PERSONAL_ACCESS_TOKEN",
        "ghcr.io/github/github-mcp-server"
      ],
    env={
        "GITHUB_PERSONAL_ACCESS_TOKEN": os.environ.get("GITHUB_PERSONAL_ACCESS_TOKEN"),
    }
)

async def ai(query: str):

    data = ""
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            tools = await load_mcp_tools(session)
            llm = ChatOpenAI(model="gpt-4.1", temperature=0)

            agent_executor = create_react_agent(llm, tools)

            # System prompt allows for broad range of instructions. But what does user input look like?
            events = agent_executor.astream(
                {"messages": [("system",
                               "You are a web server, serving HTML web pages. Do not use markdown for formatting. "
                               "All output must be in HTML format and will be displayed to an end user. "
                               "You need to start with doctype and html tags and provide the entire page"),
                              ("user", query)]},
                stream_mode="values",
            )

            async for event in events:
                event["messages"][-1].pretty_print()
                data = event["messages"][-1]

    return data.content


        

app = Flask(__name__)
    
@app.route('/', methods=['GET'])
def get_data():
    query = Path("my_app-step-3-mcp.txt").read_text()
    response = asyncio.run(ai(query))
    return response

app.run(debug=False, port=5001, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit


================================ Human Message =================================

Given my repo: https://github.com/microsoft/AI_Agents_Hackathon
Find the last 10 open issues.

For each repo make sure to display labels, project repo, and demo video link, if they exist. 

Don't make up links or any other content, just use the data provided.
================================== Ai Message ==================================
Tool Calls:
  list_issues (call_SfckEZvgYrt78Cxcfd9xTzP7)
 Call ID: call_SfckEZvgYrt78Cxcfd9xTzP7
  Args:
    owner: microsoft
    repo: AI_Agents_Hackathon
    state: open
    perPage: 10
    sort: created
    direction: desc
================================= Tool Message =================================
Name: list_issues

[{"id":3059183047,"number":760,"state":"open","locked":false,"title":"Project: Astra - Your AI Companion, Language: Python, Framework: Semantic Kernel, Azure AI Agent Service [Resubmission as previous account suspended/hacked]","body":"### Project Na

127.0.0.1 - - [16/May/2025 21:12:48] "GET / HTTP/1.1" 200 -


================================== Ai Message ==================================

<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Last 10 Open Issues - AI Agents Hackathon</title>
  <style>
    body { font-family: Arial, sans-serif; background: #f8f9fa; color: #222; }
    .issue-list { max-width: 900px; margin: 2em auto; background: #fff; border-radius: 8px; box-shadow: 0 2px 8px #0001; padding: 2em; }
    .issue { border-bottom: 1px solid #eee; padding: 1.5em 0; }
    .issue:last-child { border-bottom: none; }
    .issue-title { font-size: 1.2em; font-weight: bold; margin-bottom: 0.5em; }
    .labels { margin-bottom: 0.5em; }
    .label { display: inline-block; padding: 0.2em 0.7em; border-radius: 12px; font-size: 0.9em; margin-right: 0.4em; color: #fff; }
    .label[data-color] { background: var(--label-color, #888); }
    .meta { font-size: 0.97em; color: #555; margin-bottom: 0.5em; }
    .links { margin-top: 0.5em; }
    .links a { color: #0074d9; text-dec

127.0.0.1 - - [16/May/2025 21:17:35] "GET / HTTP/1.1" 200 -


================================== Ai Message ==================================

<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Last 10 Open Issues - AI Agents Hackathon</title>
  <style>
    body { font-family: Arial, sans-serif; background: #f8f9fa; color: #222; }
    .issue-list { max-width: 900px; margin: 2em auto; background: #fff; border-radius: 8px; box-shadow: 0 2px 8px #0001; padding: 2em; }
    .issue { border-bottom: 1px solid #eee; padding: 1.5em 0; }
    .issue:last-child { border-bottom: none; }
    .issue-title { font-size: 1.2em; font-weight: bold; margin-bottom: 0.3em; }
    .labels { margin-bottom: 0.5em; }
    .label { display: inline-block; padding: 0.2em 0.7em; border-radius: 12px; font-size: 0.9em; margin-right: 0.3em; color: #fff; }
    .label-Python { background: #d73a4a; }
    .label-JavaScript-TypeScript { background: #a2eeef; color: #222; }
    .label-Azure-AI-Agent-Service { background: #7057ff; }
    .label-Copilot { background: 

## Goals
- Write natural language only
- Unstructured data
- Implicit algorithms
- ~~Dynamic tools~~
- ~~Human in the loop built-in~~
- ~~Consistent output~~
- ~~Iterative development~~